# Pipeline pour construire une BDD et la sauvegarder dans le bucket calendrier

In [1]:
import datetime
import urllib.request
import csv
import os
import subprocess


## parametres des années à scrapper


In [2]:
yr_dbt = 16
currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
yr_fin = int(date.strftime("%y"))


## creation d'une destination local
1. verifier si le dossier existe sinon le creer temporairement
2. telecharger les fichiers data en question dans le dossier

In [3]:
s_dir="./data_F1/"
if not os.path.exists(s_dir):
    os.mkdir(s_dir)
for i in range(yr_dbt,yr_fin):
    urllib.request.urlretrieve(f"https://www.football-data.co.uk/mmz4281/{i}{i+1}/F1.csv", f"./{s_dir}F1_{i}_{i+1}.csv")
#Div	Date	Time	HomeTeam	AwayTeam	FTHG	FTAG	FTR	HTHG	HTAG	HTR	HS	AS	HST	AST	HF	AF	HC	AC	HY	AY	HR	AR	B365H	B365D	B365A	BWH	BWD	BWA	IWH	IWD	IWA	PSH	PSD	PSA	WHH	WHD	WHA	VCH	VCD	VCA	MaxH	MaxD	MaxA	AvgH	AvgD	AvgA	B365>2.5	B365<2.5	P>2.5	P<2.5	Max>2.5	Max<2.5	Avg>2.5	Avg<2.5	AHh	B365AHH	B365AHA	PAHH	PAHA	MaxAHH	MaxAHA	AvgAHH	AvgAHA	B365CH	B365CD	B365CA	BWCH	BWCD	BWCA	IWCH	IWCD	IWCA	PSCH	PSCD	PSCA	WHCH	WHCD	WHCA	VCCH	VCCD	VCCA	MaxCH	MaxCD	MaxCA	AvgCH	AvgCD	AvgCA	B365C>2.5	B365C<2.5	PC>2.5	PC<2.5	MaxC>2.5	MaxC<2.5	AvgC>2.5	AvgC<2.5	AHCh	B365CAHH	B365CAHA	PCAHH	PCAHA	MaxCAHH	MaxCAHA	AvgCAHH	AvgCAHA

## creation un seul fichier data pour toutes les données scrappées

In [4]:
merged =open(s_dir+"F1.csv","w")    
#merged.write('HomeTeam,AwayTeam,FTHG,FTAG\n')  
for yr in range(16,22):
    if yr<19 :
        f = open(f"{s_dir}F1_{yr}_{yr+1}.csv")
        freading = csv.reader(f,delimiter=',')
        next(freading) # skip the header
        for ligne in freading:
            merged.write(ligne[2]+','+ligne[3]+','+ligne[4]+','+ligne[5]+'\n')
        f.close() # not really needed
    else:
        f = open(f"{s_dir}F1_{yr}_{yr+1}.csv")
        freading = csv.reader(f,delimiter=',')
        next(freading) # skip the header
        for ligne in freading:
            merged.write(ligne[3]+','+ligne[4]+','+ligne[5]+','+ligne[6]+'\n')
        f.close() # not really needed

merged.close()

## Deplacer le BDD final (le fichier F1 avec les fichiers telechargés) à notre bucket dans le chemin destiné

In [5]:
path_local = f"{s_dir}F1.csv"
path_bucket = "gs://bucket-git-m2i/DB/calendrier"
subprocess.call(f"gsutil cp {path_local} {path_bucket}", shell=True)
for i in range(yr_dbt,yr_fin):
    subprocess.call(f"gsutil cp ./{s_dir}F1_{i}_{i+1}.csv {path_bucket}", shell=True)


Copying file://./data_F1/F1.csv [Content-Type=text/csv]...
/ [1 files][ 40.8 KiB/ 40.8 KiB]                                                
Operation completed over 1 objects/40.8 KiB.                                     
Copying file://././data_F1/F1_16_17.csv [Content-Type=text/csv]...
/ [1 files][ 92.7 KiB/ 92.7 KiB]                                                
Operation completed over 1 objects/92.7 KiB.                                     
Copying file://././data_F1/F1_17_18.csv [Content-Type=text/csv]...
/ [1 files][ 93.1 KiB/ 93.1 KiB]                                                
Operation completed over 1 objects/93.1 KiB.                                     
Copying file://././data_F1/F1_18_19.csv [Content-Type=text/csv]...
/ [1 files][ 89.7 KiB/ 89.7 KiB]                                                
Operation completed over 1 objects/89.7 KiB.                                     
Copying file://././data_F1/F1_19_20.csv [Content-Type=text/csv]...
/ [1 files][122.1 KiB

## Liberer l'espace local par supprimer le dossier creer temporairement

In [6]:
!rm -r $s_dir